# Concurrency and Parallelism
## Concurrency
**Definisi**
Concurrency adalah kemampuan suatu program untuk menangani banyak tugas secara bersama-sama (secara bergantian dalam waktu yang sangat cepat), meskipun tidak harus berjalan pada saat yang sama.
**Analogi**
Bayangkan satu koki yang sedang memasak tiga masakan. Dia berpindah-pindah tugas dengan cepat: menunggu air mendidih, lalu memotong sayur, lalu mengaduk sup. Tidak dilakukan bersamaan, tapi semua berjalan.
**Teknis**
* Biasanya menggunakan single-core CPU
* Dibantu dengan threading, asynchronous I/O, atau coroutines (seperti async dan await di Python)
* Cocok untuk I/O-bound tasks (misal: membaca file, web scraping, database query)
## Parallelism
**Definisi**
Parallelism adalah menjalankan beberapa tugas secara benar-benar bersamaan, pada waktu yang sama, dengan multiple core CPU atau multiple processor.
**Analogi**
Bayangkan tiga koki di tiga dapur yang masing-masing memasak satu masakan. Semua tugas berlangsung secara paralel dan bersamaan.
**Teknis**
* Butuh multi-core CPU
* Menggunakan proses paralel (multiprocessing, joblib, GPU processing)
* Cocok untuk CPU-bound tasks (misal: komputasi berat, image processing, training ML model)

"All parallelism is concurrent, but not all concurrency is parallel."

# Simple

Program ini menunjukkan bagaimana sebuah tugas dijalankan secara terpisah namun tetap diselaraskan agar program utama menunggu hingga tugas tersebut selesai sebelum melanjutkan eksekusi akhir.

In [3]:
import threading
import time

def tugas():
    print("Tugas dimulai...")
    time.sleep(5)
    print("Tugas selesai!")
    time.sleep(5)
    print("Tugas selesai!")

# Membuat thread
t = threading.Thread(target=tugas)

# Memulai thread
t.start()

# Menunggu thread selesai
t.join()

print("Semua tugas selesai!")


Tugas dimulai...
Tugas selesai!
Tugas selesai!
Semua tugas selesai!


Program ini menggambarkan bagaimana sebuah tugas dapat berjalan secara bersamaan dengan program utama, memungkinkan bagian lain dari program untuk terus berjalan tanpa harus menunggu tugas tersebut selesai.

In [4]:
import threading
import time

def tugas():
    print("Tugas dimulai...")
    time.sleep(5)
    print("Tugas selesai!")

# Membuat thread
t = threading.Thread(target=tugas)

# Memulai thread
t.start()

print("Semua tugas selesai!")


Tugas dimulai...Semua tugas selesai!

Tugas selesai!


Tugas selesai!


## Contoh 1: Membuat Thread untuk Eksekusi Paralel

Kode ini menggambarkan dua aktivitas yang dijalankan secara bersamaan menggunakan dua thread terpisah: satu mencetak angka dari 1 hingga 5, dan yang lain mencetak huruf dari ‘a’ hingga ‘e’. Keduanya berjalan serempak, memungkinkan angka dan huruf muncul secara berselang-seling tergantung pada kecepatan eksekusi masing-masing thread. Program utama menunggu hingga kedua thread selesai sebelum mengakhiri eksekusi.

In [5]:
import threading
import time

def print_numbers():
    for i in range(1, 6):
        print(f"Number: {i}")
        time.sleep(1)

def print_letters():
    for letter in 'abcde':
        print(f"Letter: {letter}")
        time.sleep(1)

thread1 = threading.Thread(target=print_numbers)
thread2 = threading.Thread(target=print_letters)

thread1.start()
thread2.start()

thread1.join()
thread2.join()


Number: 1
Letter: a
Number: 2Letter: b

Letter: c
Number: 3
Letter: dNumber: 4

Letter: eNumber: 5



Penjelasan:
* Dalam contoh ini, terdapat dua fungsi, print_numbers dan print_letters, masing-masing mensimulasikan sebuah tugas.
* Kami membuat dua thread, thread1 dan thread2, untuk menjalankan fungsi-fungsi ini secara bersamaan.
* thread1.start() dan thread2.start() memulai thread-thread tersebut.
* Kami menggunakan thread1.join() dan thread2.join() untuk menunggu thread selesai.
* Thread-thread ini menjalankan tugas secara bersamaan, mencetak angka dan huruf, dan melakukan sleep selama 1 detik setelah mencetak setiap karakter.

## Contoh 2: Keamanan Thread dengan Sumber Daya Bersama
Kode ini mendemonstrasikan bagaimana dua thread dapat bekerja bersama untuk menambah nilai suatu variabel bersama (counter) sebanyak 100.000 kali masing-masing, sambil menghindari konflik atau kesalahan akibat akses bersamaan. Dengan menggunakan lock, kode memastikan bahwa hanya satu thread yang dapat memodifikasi counter pada satu waktu, menjaga akurasi hasil akhir. Setelah kedua thread selesai dijalankan, program mencetak nilai akhir dari counter, yang seharusnya bernilai 200.000 jika semua operasi berhasil dijalankan tanpa konflik.

In [6]:
import threading

counter = 0
lock = threading.Lock()

def increment():
    global counter
    for _ in range(100000):
        with lock:
            counter += 1

thread1 = threading.Thread(target=increment)
thread2 = threading.Thread(target=increment)

thread1.start()
thread2.start()

thread1.join()
thread2.join()

print("Counter:", counter)


Counter: 200000


Penjelasan:

Contoh ini menggambarkan masalah umum dalam pemrograman konkurensi: sumber daya bersama.
Kami memiliki variabel global counter yang diinkremen oleh beberapa thread secara bersamaan.
Untuk memastikan keamanan thread, kami menggunakan threading.Lock untuk melindungi bagian kritis di mana counter diperbarui.
Setiap thread, thread1 dan thread2, menginkremen counter sebanyak 100,000 kali.
Setelah kedua thread selesai, kami mencetak nilai terakhir dari counter. Menggunakan lock memastikan bahwa counter diperbarui secara aman.

# Intermediate

## Contoh 1: Skenario Produsen-Konsumen dengan Thread
Kode ini menggambarkan skenario produsen-konsumen menggunakan dua thread dan struktur antrian (queue.Queue) untuk komunikasi antar thread. Thread produsen menghasilkan lima item dan menempatkannya ke dalam antrian, sedangkan thread konsumen mengambil item satu per satu dari antrian dan mencetaknya sebagai tanda bahwa item tersebut telah "dikonsumsi". Setelah produsen selesai, ia mengirimkan sinyal khusus (None) ke konsumen sebagai tanda bahwa tidak ada lagi item yang akan diproduksi, sehingga thread konsumen dapat berhenti secara bersih. Pendekatan ini menjaga sinkronisasi dan alur kerja antara dua thread tanpa konflik.

In [7]:
import threading
import queue

def produsen(q):
    for i in range(5):
        q.put(i)

def konsumen(q):
    while True:
        item = q.get()
        if item is None:
            break
        print("Dikonsumsi:", item)

q = queue.Queue()
thread_produsen = threading.Thread(target=produsen, args=(q,))
thread_konsumen = threading.Thread(target=konsumen, args=(q,))

thread_produsen.start()
thread_konsumen.start()

thread_produsen.join()
q.put(None)  # Sinyal kepada konsumen untuk berhenti
thread_konsumen.join()


Dikonsumsi: 0
Dikonsumsi: 1
Dikonsumsi: 2
Dikonsumsi: 3
Dikonsumsi: 4


Penjelasan:
* Dalam contoh ini, kami memiliki dua thread yang mewakili produsen dan konsumen.
* Thread produsen, thread_produsen, menghasilkan dan menambahkan item ke antrian bersama.
* Thread konsumen, thread_konsumen, mengonsumsi item dari antrian.
* Thread konsumen terus mengonsumsi item hingga menerima sinyal "berhenti" (None) dari produsen.
* Penggunaan antrian memastikan keselamatan konkurensi dan mengizinkan komunikasi yang aman antara produsen dan konsumen.

## Contoh 2: Thread Pool untuk Paralelisme Tugas
Kode ini menunjukkan penggunaan ThreadPoolExecutor dari modul concurrent.futures untuk menjalankan fungsi secara paralel menggunakan thread. Fungsi kuadrat menghitung kuadrat dari suatu angka setelah jeda satu detik. Daftar angka [1, 2, 3, 4, 5] diproses secara bersamaan oleh maksimal tiga thread yang berjalan paralel. Dengan pendekatan ini, eksekusi total menjadi lebih cepat dibandingkan menjalankannya satu per satu, karena beberapa tugas dijalankan secara bersamaan. Setelah semua hasil dihitung, program mencetak daftar hasil kuadratnya.

In [8]:
from concurrent.futures import ThreadPoolExecutor
import time

def kuadrat(x):
    time.sleep(1)
    return x * x

data = [1, 2, 3, 4, 5]
with ThreadPoolExecutor(max_workers=3) as executor:
    hasil = list(executor.map(kuadrat, data))

print("Hasil:", hasil)


Hasil: [1, 4, 9, 16, 25]


Penjelasan:\
Pada contoh ini, kami memperkenalkan ThreadPoolExecutor dari modul concurrent.futures, yang menyediakan cara praktis untuk mengelola dan menjalankan tugas secara bersamaan.
Kami mendefinisikan fungsi kuadrat yang mensimulasikan tugas dengan melakukan kuadrat angka dan tidur selama 1 detik.
Kami membuat daftar data dan menggunakan metode executor.map untuk pengeksekusian secara bersamaan

# Advanced

## Contoh 1: Eksekusi Bersamaan dengan ThreadPoolExecutor
Kode ini merupakan implementasi concurrent programming menggunakan ThreadPoolExecutor dari modul concurrent.futures untuk menjalankan fungsi secara paralel dengan thread. Fungsi kuadrat akan menghitung kuadrat dari setiap angka dalam daftar data yang berisi angka 0 hingga 99, dengan jeda satu detik setiap eksekusi. Karena max_workers=3, maka hanya tiga tugas akan dijalankan secara bersamaan dalam satu waktu oleh tiga thread yang aktif. Meskipun setiap perhitungan butuh waktu, pendekatan ini mempercepat proses total dibandingkan jika dilakukan secara berurutan. Setelah semua kuadrat selesai dihitung secara paralel, hasilnya dikumpulkan dan ditampilkan sebagai daftar.

In [9]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import time

def kuadrat(x):
    time.sleep(1)
    return x * x

data = [i for i in range(100)]
with ThreadPoolExecutor(max_workers=3) as executor:
    hasil = list(executor.map(kuadrat, data))

print("Hasil (Thread):", hasil)


Hasil (Thread): [0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361, 400, 441, 484, 529, 576, 625, 676, 729, 784, 841, 900, 961, 1024, 1089, 1156, 1225, 1296, 1369, 1444, 1521, 1600, 1681, 1764, 1849, 1936, 2025, 2116, 2209, 2304, 2401, 2500, 2601, 2704, 2809, 2916, 3025, 3136, 3249, 3364, 3481, 3600, 3721, 3844, 3969, 4096, 4225, 4356, 4489, 4624, 4761, 4900, 5041, 5184, 5329, 5476, 5625, 5776, 5929, 6084, 6241, 6400, 6561, 6724, 6889, 7056, 7225, 7396, 7569, 7744, 7921, 8100, 8281, 8464, 8649, 8836, 9025, 9216, 9409, 9604, 9801]


Kode ini menunjukkan pemrosesan paralel menggunakan ThreadPoolExecutor dengan jumlah worker yang besar (max_workers=100). Fungsi kuadrat menghitung kuadrat dari angka dalam daftar data yang berisi bilangan 0 hingga 99, dan setiap pemrosesan diberi jeda satu detik untuk mensimulasikan proses yang memakan waktu. Karena jumlah thread sebanyak 100 sama dengan jumlah elemen dalam data, seluruh tugas dapat diproses secara bersamaan, sehingga waktu total eksekusi akan mendekati waktu eksekusi satu fungsi saja (sekitar 1 detik), bukan 100 detik seperti pada pemrosesan berurutan. Pendekatan ini memperlihatkan kekuatan concurrency dalam memanfaatkan thread secara maksimal untuk meningkatkan efisiensi, meskipun dalam praktiknya jumlah thread yang terlalu banyak bisa memberi tekanan pada sistem tergantung pada beban dan sumber daya.

In [10]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import time

def kuadrat(x):
    time.sleep(1)
    return x * x

data = [i for i in range(100)]
with ThreadPoolExecutor(max_workers=100) as executor:
    hasil = list(executor.map(kuadrat, data))

print("Hasil (Thread):", hasil)

Hasil (Thread): [0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361, 400, 441, 484, 529, 576, 625, 676, 729, 784, 841, 900, 961, 1024, 1089, 1156, 1225, 1296, 1369, 1444, 1521, 1600, 1681, 1764, 1849, 1936, 2025, 2116, 2209, 2304, 2401, 2500, 2601, 2704, 2809, 2916, 3025, 3136, 3249, 3364, 3481, 3600, 3721, 3844, 3969, 4096, 4225, 4356, 4489, 4624, 4761, 4900, 5041, 5184, 5329, 5476, 5625, 5776, 5929, 6084, 6241, 6400, 6561, 6724, 6889, 7056, 7225, 7396, 7569, 7744, 7921, 8100, 8281, 8464, 8649, 8836, 9025, 9216, 9409, 9604, 9801]


Penjelasan:

Pada contoh ini, kami menggunakan ThreadPoolExecutor untuk mengeksekusi fungsi kuadrat secara bersamaan dengan maksimal 3 thread.

Fungsi kuadrat mensimulasikan tugas dengan mengkuadratkan angka dan tidur selama 1 detik.

Metode executor.map memetakan fungsi ke data dan hasilnya dikumpulkan.

Ini menggambarkan eksekusi bersamaan dengan thread.

## Contoh 2: Eksekusi Bersamaan dengan ProcessPoolExecutor
Kode ini menggunakan pendekatan parallelism melalui ProcessPoolExecutor untuk mengeksekusi fungsi kuadrat secara paralel menggunakan proses-proses terpisah. Fungsi kuadrat akan mengkuadratkan tiap elemen dalam daftar data, namun dengan penundaan selama 1 detik untuk mensimulasikan tugas yang berat. Karena max_workers=3, maka hanya tiga proses yang berjalan bersamaan. Sisanya akan menunggu giliran hingga ada proses yang selesai. Hal ini membuat eksekusi menjadi lebih cepat dibanding secara berurutan, namun tetap terbatas pada jumlah worker yang tersedia. Karena menggunakan proses (bukan thread), pendekatan ini cocok untuk tugas-tugas CPU-bound yang memerlukan performa lebih tinggi dan dapat memanfaatkan banyak inti CPU secara efisien. Hasil akhir dari perhitungan kuadrat seluruh elemen ditampilkan setelah semua proses selesai.

In [11]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import time

def kuadrat(x):
    time.sleep(1)
    return x * x

data = [1, 2, 3, 4, 5, 10]
with ProcessPoolExecutor(max_workers=3) as executor:
    hasil = list(executor.map(kuadrat, data))

print("Hasil (Proses):", hasil)

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [12]:
from concurrent.futures import ProcessPoolExecutor
import time

def kuadrat(x):
    time.sleep(1)
    return x * x

if __name__ == "__main__":
    data = [1, 2, 3, 4, 5, 10]
    with ProcessPoolExecutor(max_workers=3) as executor:
        hasil = list(executor.map(kuadrat, data))

    print("Hasil (Proses):", hasil)

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

Penjelasan:

Contoh ini mirip dengan yang sebelumnya, tetapi menunjukkan eksekusi bersamaan menggunakan proses alih-alih thread.
Kami menggunakan ProcessPoolExecutor untuk mengeksekusi fungsi kuadrat dengan maksimal 3 proses.
Fungsi kuadrat mensimulasikan tugas dengan mengkuadratkan angka dan tidur selama 1 detik.
Metode executor.map memetakan fungsi ke data dan hasilnya dikumpulkan.
Eksekusi berbasis proses cocok untuk tugas yang membutuhkan banyak sumber daya CPU.

Kode di bawah menunjukkan proses pengunduhan lima gambar dari internet secara berurutan menggunakan modul requests. Setiap gambar diunduh satu per satu, disimpan ke file lokal, dan waktu total proses diukur menggunakan modul time. Pendekatan ini bersifat sekuensial, sehingga durasi eksekusi akan bertambah seiring banyaknya gambar yang diunduh.

In [13]:
import requests
import time

urls = [
    "https://picsum.photos/200/300",  # Gambar 1
    "https://picsum.photos/200/301",  # Gambar 2
    "https://picsum.photos/200/302",  # Gambar 3
    "https://picsum.photos/200/303",  # Gambar 4
    "https://picsum.photos/200/304",  # Gambar 5
]

def download_image(url, index):
    response = requests.get(url)
    with open(f"gambar_{index}.jpg", "wb") as file:
        file.write(response.content)
    print(f"Gambar {index} selesai diunduh.")

start_time = time.time()
for i, url in enumerate(urls):
    download_image(url, i)

print(f"Selesai dalam {time.time() - start_time:.2f} detik")

Gambar 0 selesai diunduh.
Gambar 1 selesai diunduh.
Gambar 2 selesai diunduh.
Gambar 3 selesai diunduh.
Gambar 4 selesai diunduh.
Selesai dalam 36.29 detik


Kode ini menunjukkan proses pengunduhan lima gambar dari internet secara paralel menggunakan ThreadPoolExecutor. Dengan memanfaatkan tiga thread sekaligus, beberapa gambar dapat diunduh secara bersamaan, sehingga waktu total eksekusi menjadi lebih efisien dibandingkan pendekatan sekuensial.

In [14]:
from concurrent.futures import ThreadPoolExecutor
import requests
import time

urls = [
    "https://picsum.photos/200/300",
    "https://picsum.photos/200/301",
    "https://picsum.photos/200/302",
    "https://picsum.photos/200/303",
    "https://picsum.photos/200/304",
]

def download_image(url, index):
    response = requests.get(url)
    with open(f"gambar_{index}.jpg", "wb") as file:
        file.write(response.content)
    print(f"Gambar {index} selesai diunduh.")

start_time = time.time()
with ThreadPoolExecutor(max_workers=3) as executor:
    executor.map(download_image, urls, range(len(urls)))

print(f"Selesai dalam {time.time() - start_time:.2f} detik")


Gambar 1 selesai diunduh.
Gambar 0 selesai diunduh.
Gambar 3 selesai diunduh.
Gambar 4 selesai diunduh.
Gambar 2 selesai diunduh.
Selesai dalam 9.81 detik


Kode ini berusaha mengunduh lima gambar secara paralel menggunakan ProcessPoolExecutor, yang menjalankan tugas pada beberapa proses terpisah. Meskipun secara teori cocok untuk tugas CPU-bound, penggunaan requests (yang tidak dapat dipickle secara langsung) dalam multiprocessing bisa menyebabkan error. Tujuan utamanya adalah mempercepat proses unduh dengan memanfaatkan pemrosesan paralel berbasis proses.







In [15]:
from concurrent.futures import ProcessPoolExecutor
import requests
import time

urls = [
    "https://picsum.photos/200/300",
    "https://picsum.photos/200/301",
    "https://picsum.photos/200/302",
    "https://picsum.photos/200/303",
    "https://picsum.photos/200/304",
]

def download_image(url, index):
    response = requests.get(url)
    with open(f"gambar_{index}.jpg", "wb") as file:
        file.write(response.content)
    print(f"Gambar {index} selesai diunduh.")

start_time = time.time()
with ProcessPoolExecutor(max_workers=3) as executor:
    executor.map(download_image, urls, range(len(urls)))

print(f"Selesai dalam {time.time() - start_time:.2f} detik")


Selesai dalam 0.26 detik


Percobaan menunjukkan bahwa pengunduhan gambar satu per satu (sekuensial) membutuhkan waktu paling lama, yaitu sekitar 36.29 detik. Penggunaan ThreadPoolExecutor mampu memangkas waktu secara signifikan menjadi sekitar 9.81 detik karena memungkinkan beberapa unduhan berjalan secara bersamaan dalam satu proses menggunakan beberapa thread. Menariknya, ProcessPoolExecutor mencatat waktu tercepat, sekitar 0.26 detik, meskipun secara umum lebih cocok untuk tugas berat CPU (bukan I/O seperti requests). Namun, hasil ini bisa jadi tidak konsisten karena proses paralel bisa dipengaruhi oleh cache, jaringan, atau lingkungan sistem.

Secara praktis, untuk tugas I/O-bound seperti mengunduh dari internet, ThreadPoolExecutor adalah pilihan yang lebih aman dan efisien. Hasil ProcessPoolExecutor yang sangat cepat patut diuji ulang karena bisa saja terjadi anomali sistem atau caching internal.